In [ ]:
import librosa
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import BeatNet
import numpy as np
from BeatNet.BeatNet import BeatNet
from madmom.features.tempo import TempoEstimationProcessor
from madmom.features.beats import RNNBeatProcessor
import essentia.standard as es
import pandas as pd


In [ ]:
fps = 100
sr = 22050

In [ ]:
def tempo_librosa(y,sr):
    return librosa.feature.tempo(y = y, sr=sr)

In [ ]:
def tempo_essentia(y):
    rhythm_extractor = es.RhythmExtractor2013(method="multifeature")
    tempo, _,_, _, _ = rhythm_extractor(y)
    return tempo

In [ ]:
def tempo_madmom(y):
    bt_prob_fn1 = RNNBeatProcessor()
    bt_prob1 = bt_prob_fn1(y)
    tmp_est_proc = TempoEstimationProcessor(method = 'dbn', fps = 100)
    tempo1 = tmp_est_proc(bt_prob1)

    def find_max_in_second_column(matrix):
        max_value = float('-inf')
        corresponding_element = None
    
        for row in matrix:
            if row[1] > max_value:
                max_value = row[1]
                corresponding_element = row[0]
        return corresponding_element
    tempo = find_max_in_second_column(tempo1)
    # print('Estimated Tempo:',tempo)
    return tempo

In [ ]:
def tempo_beatnet(y):
    estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)
    Output1= estimator.process(y)
    original_array1 = Output1[:,0]
    if len(original_array1 ==1):
        if original_array1[0]==0:
            tempo = 60/10000
        else:
            tempo = 60/original_array1[0]
    else:
        differences1 = [original_array1[i+1] - original_array1[i] for i in range(len(original_array1)-1)]
        tempo = [60 / differences1[-1]]
    return tempo

In [ ]:
sensor_input_file =r'sensor input csv file'
data_frame1 = pd.read_csv(sensor_input_file)
sensor_input = data_frame1['Index']

In [ ]:
def sensor_2_audio(sensor_input):
    activ_arr = []
    for i in (range(len(sensor_input))):
        if(sensor_input[i]!=0):
            activ_arr.append(1)
        else:
            activ_arr.append(0)

    flag = 0
    for i in (range(0,len(activ_arr))):
        if (activ_arr[i]==1 and flag==0):
            flag = 1
        elif (activ_arr[i]==1 and flag==1):
            activ_arr[i] = 0
        elif (activ_arr[i]==0 and flag==1):
            flag = 0
            
    activ_arr = np.array(activ_arr, float)
    activ_arr = librosa.resample(activ_arr, orig_sr = 200, target_sr = 22050)
    return activ_arr

In [ ]:
input_to_tempo_est= sensor_2_audio(sensor_input)
frames_lr= librosa.util.frame(input_to_tempo_est, frame_length=10*sr, hop_length=350)

In [ ]:
tempo_lr = []
tempo_mm = []
tempo_es = []
tempo_btn = []

for i in tqdm(range(range(len(frames_lr[0,:])))):
    tempo_lr.append(tempo_librosa(y = frames_lr[:,i], sr=sr))
    tempo_es.append(tempo_essentia(frames_lr[:,i]))
    tempo_mm.append(tempo_madmom(frames_lr[:,i]))
    tempo_btn.append(tempo_beatnet(frames_lr[:,i]))

